In [7]:
# !pip install sqlalchemy


In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [9]:
# !ln -s ../../stockdata .
# !ln -s ../../data .

In [2]:
# engine = create_engine('sqlite:///NQ_DB.db')
engine = create_engine('sqlite:///data/ESNQ_DB.db')

In [3]:

table_names = inspect(engine).get_table_names()

print("Tables in the database:")
table_names

Tables in the database:


['ES1D',
 'ES1M',
 'ES30M',
 'ES4H',
 'ES5M',
 'NQ1D',
 'NQ1M',
 'NQ30M',
 'NQ4H',
 'NQ5M']

In [64]:
import glob
ticks = ['data/ES*.csv', 'data/NQ*.csv']
# ticks = ['data/DAT*.csv']
files = []
for tick in ticks:
    files.append(glob.glob(tick))

In [60]:
files = !ls data/ES*.csv
# files = !ls data/NQ*.csv
print(files)

['zsh:1: no matches found: data/ES*.csv']


In [65]:
# files = !ls stockdata/ES*30M.csv
# files = !ls data/ES*30M.csv
for file in files:
    print(file)


[]
[]


In [49]:
def build_dict(file_list):
    result = {}
    for file_path in file_list:
        # Extract the filename from the path
        file_name = file_path.split('/')[-1]
        # Split the filename and extract the ticker and timeframe (e.g., ES_1D or ES_30M)
        ticker_timeframe = file_name.split('_')[0] + file_name.split('_')[-1].replace(".csv", "")
        # Map the file path to the desired value
        result[file_path] = ticker_timeframe
    return result

# Example usage
# file_list = [
#     "data/ES_1997-09-08_2024-09-19_1D.csv",
#     "data/ES_2024-01-01_2024-09-20_30M.csv"
# ]


In [50]:
outputDic = build_dict(files)
print(outputDic)

{'data/ES_1997-09-08_2024-10-03_1D.csv': 'ES1D', 'data/ES_2021-01-03_2024-10-04_4H.csv': 'ES4H', 'data/ES_2024-01-01_2024-10-04_30M.csv': 'ES30M', 'data/ES_2024-09-08_2024-10-04_5M.csv': 'ES5M', 'data/ES_2024-09-29_2024-10-04_1M.csv': 'ES1M'}


In [51]:
for key, value in outputDic.items():
    print(f"Key: {key}, Value: {value}")


Key: data/ES_1997-09-08_2024-10-03_1D.csv, Value: ES1D
Key: data/ES_2021-01-03_2024-10-04_4H.csv, Value: ES4H
Key: data/ES_2024-01-01_2024-10-04_30M.csv, Value: ES30M
Key: data/ES_2024-09-08_2024-10-04_5M.csv, Value: ES5M
Key: data/ES_2024-09-29_2024-10-04_1M.csv, Value: ES1M


In [52]:
def sql_importer (df, table_name):
    # Use inspector to check if the table exists
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        df.to_sql(table_name, engine, if_exists='append')
        print (f'New table created for {table_name} with {str(len(df))} rows')
    else:
        max_date = pd.read_sql(f'SELECT MAX(datetime) FROM {table_name}', engine).values[0][0]
        print(max_date)
        df = df[df.datetime > max_date]
        df.to_sql(table_name, engine, if_exists='append')
        print(str(len(df)) + ' new rows imported to DB ' + f'{table_name}')

In [53]:
for key, value in outputDic.items():
    df = pd.read_csv(key)
    sql_importer(df, value)

2024-09-26 17:00:00
5 new rows imported to DB ES1D
2024-09-27 13:00:00
30 new rows imported to DB ES4H
2024-09-27 15:30:00
230 new rows imported to DB ES30M
2024-09-27 15:55:00
1380 new rows imported to DB ES5M
2024-09-27 15:59:00
6900 new rows imported to DB ES1M


In [54]:
# for file in files:
#     df = pd.read_csv(file)
#     sql_importer(df, 'ES30M')

In [55]:
pd.read_sql('ES30M', engine)

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-01-01 17:00:00,CME_MINI:ES1!,4818.00,4822.50,4815.75,4821.75,8431.0
1,1,2024-01-01 17:30:00,CME_MINI:ES1!,4822.00,4822.25,4820.25,4821.00,4871.0
2,2,2024-01-01 18:00:00,CME_MINI:ES1!,4820.75,4821.50,4819.00,4819.25,2533.0
3,3,2024-01-01 18:30:00,CME_MINI:ES1!,4819.25,4820.50,4818.75,4819.75,2075.0
4,4,2024-01-01 19:00:00,CME_MINI:ES1!,4819.75,4822.75,4819.75,4821.25,8181.0
...,...,...,...,...,...,...,...,...
9048,9048,2024-10-04 13:30:00,CME_MINI:ES1!,5778.25,5786.00,5775.75,5785.50,34321.0
9049,9049,2024-10-04 14:00:00,CME_MINI:ES1!,5785.25,5792.75,5781.75,5791.25,43453.0
9050,9050,2024-10-04 14:30:00,CME_MINI:ES1!,5791.00,5804.75,5789.50,5799.75,122255.0
9051,9051,2024-10-04 15:00:00,CME_MINI:ES1!,5799.75,5801.00,5797.50,5799.00,39369.0


In [5]:
df = pd.read_sql('SELECT * FROM NQ30M', con=engine)
df

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-01-01 17:00:00,CME_MINI:NQ1!,17019.00,17037.00,17013.75,17033.25,3466.0
1,1,2024-01-01 17:30:00,CME_MINI:NQ1!,17033.25,17036.25,17027.25,17029.50,1116.0
2,2,2024-01-01 18:00:00,CME_MINI:NQ1!,17030.00,17030.75,17018.50,17019.75,1222.0
3,3,2024-01-01 18:30:00,CME_MINI:NQ1!,17019.75,17028.00,17018.50,17027.00,1048.0
4,4,2024-01-01 19:00:00,CME_MINI:NQ1!,17026.75,17038.50,17026.25,17029.25,2664.0
...,...,...,...,...,...,...,...,...
8358,8358,2024-09-13 13:30:00,CME_MINI:NQ1!,19513.00,19531.25,19496.50,19513.25,17398.0
8359,8359,2024-09-13 14:00:00,CME_MINI:NQ1!,19513.25,19538.50,19492.00,19528.75,17214.0
8360,8360,2024-09-13 14:30:00,CME_MINI:NQ1!,19529.50,19552.00,19508.75,19527.25,27066.0
8361,8361,2024-09-13 15:00:00,CME_MINI:NQ1!,19527.50,19529.00,19501.50,19522.00,8917.0


In [9]:
df = pd.read_sql('SELECT datetime, open, close FROM NQ5M', con=engine)
df

,datetime,open,close
0,2024-05-12 17:00:00,18238.00,18241.75
1,2024-05-12 17:05:00,18241.75,18230.75
2,2024-05-12 17:10:00,18230.75,18234.00
3,2024-05-12 17:15:00,18233.75,18239.00
4,2024-05-12 17:20:00,18238.50,18241.00
...,...,...,...
24598,2024-09-13 15:35:00,19518.00,19520.25
24599,2024-09-13 15:40:00,19520.00,19516.75
24600,2024-09-13 15:45:00,19516.75,19508.50
24601,2024-09-13 15:50:00,19507.50,19507.00


In [10]:
# df.describe().style.background_gradient(cmap='coolwarm')
df.describe()
df.describe().style.background_gradient(cmap='coolwarm')

,open,close
count,24603.000000,24603.000000
mean,19391.005091,19391.045838
std,705.161186,705.105002
min,17426.000000,17425.250000
25%,18774.500000,18774.500000
50%,19327.250000,19327.250000
75%,19951.250000,19951.125000
max,20972.250000,20971.750000


In [20]:
import pandas_ta as ta
macd = df.ta.macd(fast=13, slow=26)
macd

,MACD_13_26_9,MACDh_13_26_9,MACDs_13_26_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
21886,26.866794,0.975748,25.891046
21887,25.752240,-0.111044,25.863285
21888,24.337227,-1.220846,25.558073
21889,22.956664,-2.081128,25.037791


In [21]:
rsi = df.ta.rsi()
rsi

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
21886    67.754123
21887    66.532842
21888    64.447768
21889    64.447768
21890    63.524232
Name: RSI_14, Length: 21891, dtype: float64

In [23]:
from sqlalchemy import create_engine, MetaData

# Setup your engine
# engine = create_engine('your_database_url')
# engine = create_engine('sqlite:///TEST_DB.db')
engine = create_engine('sqlite:///data/ESNQ_DB.db')

# Reflect the database schema to get all table names
metadata = MetaData()
metadata.reflect(bind=engine)

print("Tables in the database:")
for table_name in metadata.tables.keys():
    print(table_name)

Tables in the database:
ES1M
ES30M
ES5M
NQ1M
NQ30M
NQ5M
